In [1]:
%load_ext autoreload
%autoreload 2

import sys; sys.path.insert(0, '..')
import json

import tweepy
from tqdm import tqdm, trange

from paradeller.keys import (
    consumer_key, consumer_secret, access_token, accss_token_secret
)
from paradeller.stopwords import stopwords

## Scraping

In [3]:
from paradeller.scrape import api, get_tweets, display_status, format_status

In [6]:
res = get_tweets()

for status in res:
    display_status(status)

Blows my mind the things people complain about.
-- @sidneypotts11

1144647813768372225
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Its too hot , i dont even wan smoke
-- @juggtoven

1144647813688676352
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
He really threw Juvenille on this mug 😂👏🏾
-- @Soul_yup

1144647813122416640
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
&gt;:( 1 point off.
-- @Niki_Yanagi

1144647812929470466
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
halo comeback when
-- @hwitaekthewheel

1144647812702973954
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
did mnet fk up again why is it trending
-- @snowinthetae

1144647812686069760
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Very entertaining but it's over now
-- @JOMMELIZA

1144647812669308928
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


## Data Collection

In [7]:
from paradeller.helper import load_archive, update_archive

In [9]:
archive = load_archive()
len(archive)

17933

## Pre-select tweets

In [9]:
def save_status_from_id(id_):
    status = api.get_status(id_)
    return save_status(status)

In [10]:
ids = [
    1144127521929224197,
    1144135638721036288,
    1144135638733557760,
    1144137985971183616,
    1144144954219999232,
    1144146074321317888,
    1144313057377546240,
    1144313057444847621
]

In [11]:
saved = []
for id_ in ids:
    try:
        s = save_status_from_id(id_)
        saved.append(s)
    except tweepy.TweepError as e:
        print(e)
        print(">> ", id_)

In [12]:
saved

[{'id': 1144127521929224197,
  'text': 'Sitting in lingerie drinking wine ima wife for sure 😭',
  'author': '__mxvii',
  'time': datetime.datetime(2019, 6, 27, 6, 17, 12)},
 {'id': 1144135638721036288,
  'text': 'time will be frozen for us',
  'author': 'mariuuhhhh',
  'time': datetime.datetime(2019, 6, 27, 6, 49, 27)},
 {'id': 1144135638733557760,
  'text': 'Is it me or is it a break up season?',
  'author': 'Linnneyyy',
  'time': datetime.datetime(2019, 6, 27, 6, 49, 27)},
 {'id': 1144137985971183616,
  'text': 'ive been gettin horny at the most inconvenient times lately wtf 🥴😫',
  'author': 'SoCvsh',
  'time': datetime.datetime(2019, 6, 27, 6, 58, 47)},
 {'id': 1144144954219999232,
  'text': 'Fuck work experience 🖕🏾🖕🏾',
  'author': 'juggz_19',
  'time': datetime.datetime(2019, 6, 27, 7, 26, 28)},
 {'id': 1144146074321317888,
  'text': 'All you can do is watch.',
  'author': 'ayeroven',
  'time': datetime.datetime(2019, 6, 27, 7, 30, 55)},
 {'id': 1144313057377546240,
  'text': 'Pier

## Searching for matches

In [13]:
import emoji
import string
from tqdm import tqdm, trange

In [14]:
def tokenize(tweet):
    tweet = ' '.join(emoji.get_emoji_regexp().split(tweet))
    words = tweet.split()
    return [
        w.lower().strip().translate(str.maketrans('', '', string.punctuation))
        for w in words
    ]

In [15]:
tokenize(saved[0]['text'])

['sitting',
 'in',
 'lingerie',
 'drinking',
 'wine',
 'ima',
 'wife',
 'for',
 'sure',
 '😭']

### Pick some tweets

In [16]:
s1 = saved[0]
s1

{'id': 1144127521929224197,
 'text': 'Sitting in lingerie drinking wine ima wife for sure 😭',
 'author': '__mxvii',
 'time': datetime.datetime(2019, 6, 27, 6, 17, 12)}

In [17]:
s2 = saved[7]
s2

{'id': 1144313057444847621,
 'text': 'did I mention i’m gay?',
 'author': 'hoeslynn',
 'time': datetime.datetime(2019, 6, 27, 18, 34, 27)}

In [18]:
word_set = set(tokenize(s1['text']) + tokenize(s2['text']))

In [19]:
word_set

{'did',
 'drinking',
 'for',
 'gay',
 'i',
 'ima',
 'in',
 'i’m',
 'lingerie',
 'mention',
 'sitting',
 'sure',
 'wife',
 'wine',
 '😭'}

In [20]:
search_words = word_set - set(stopwords)
search_words

{'did',
 'drinking',
 'gay',
 'i',
 'ima',
 'i’m',
 'lingerie',
 'mention',
 'sitting',
 'sure',
 'wife',
 'wine',
 '😭'}

In [31]:
search_words

{'did',
 'drinking',
 'gay',
 'i',
 'ima',
 'i’m',
 'lingerie',
 'mention',
 'sitting',
 'sure',
 'wife',
 'wine',
 '😭'}

In [32]:
search_words

{'did',
 'drinking',
 'gay',
 'i',
 'ima',
 'i’m',
 'lingerie',
 'mention',
 'sitting',
 'sure',
 'wife',
 'wine',
 '😭'}

In [21]:
query = (
    ' OR '.join(search_words) + 
    " -filter:retweets -filter:links -filter:media"
)
print(query)

def get_search_tweets():
    tweets = api.search(
        q=query, lang='en', count=1000, include_entities=False
    )
    return [t for t in tweets if is_good(t.text)]

sure OR did OR 😭 OR mention OR sitting OR wine OR i OR wife OR lingerie OR ima OR drinking OR gay OR i’m -filter:retweets -filter:links -filter:media


In [25]:
def potential_match(tweet, word_set):
    """
    If all words in tweet are in the word set,
    it is a potential match
    """
    words = tokenize(tweet)
    return set(words) <= word_set

In [34]:
iterations = 1000

matches = []

for i in trange(iterations):
    res = get_search_tweets()
    for status in res:
        if potential_match(status.text, word_set):
            print("yeah!!")
            display_status(status)
            mactches.append(status)

100%|██████████| 1000/1000 [1:09:07<00:00,  1.36it/s]   


In [36]:
matches

[]